In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

import couponProjectUtils as proj_utils

from sklearn.preprocessing import StandardScaler

from imblearn.over_sampling import SMOTE

## models
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score

## to make it possible to display multiple output inside one cell 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

pd.options.display.max_rows = 50
pd.set_option('display.float_format', lambda x: '%9.8f' % x)

# Use best encoding for coupon type: Bar

In [2]:
## load data, split Xy, upsample with SMOTE, split train/test
data = pd.read_csv('data/Bar_data_encoded-all5_nominal.csv')

## define X and y 
X = data.drop('Y', axis=1).reset_index(drop=True)
y = data.Y

## Handle imbalance
smote = SMOTE(random_state=42)

orig_vc = y.value_counts()
print(f"Orig: {100* orig_vc[1]/(orig_vc.sum()):.2f}% Yes")

X_sm, y_sm = smote.fit_resample(X, y)

## Data splitting train/test
X_train, X_test, y_train, y_test = train_test_split(X_sm, y_sm, test_size=0.20, random_state=42)

print("After SMOTE upsampling")
train_vc = y_train.value_counts()
print(f"\tNum training samples: {X_train.shape[0]:,} ({100* train_vc[1]/(train_vc.sum()):.2f}% yes)")
test_vc = y_test.value_counts()
print(f"\tNum testing samples:  {X_test.shape[0]:,} ({100* test_vc[1]/(test_vc.sum()):.2f}% yes)")

Orig: 41.00% Yes
After SMOTE upsampling
	Num training samples: 1,897 (50.82% yes)
	Num testing samples:  475 (46.74% yes)


In [3]:
## Replace the unknown values (previously encoded as 0.0) using the most frequent value seen in the TRAINING data
X_train, X_test, train_modes = proj_utils.replace_unknown_with_mode(X_train, X_test, outfile_prefix='phase2-Bar')

## Testing RandomForestClassifier with Grid Search Cross-validation

Based on initial testing, the following params will be left out of the search (their default values was always the best setting):
* min_samples_leaf (1)
* max_depth (None)
* max_leaf_nodes (None)

In [4]:
cols_to_view = ['rank_test_score', 'param_max_features', 'param_min_samples_split', 'param_n_estimators', 
                'mean_test_score', 'std_test_score', 'mean_train_score', 'std_train_score']

In [5]:
param_grid={'max_features': [5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
            'min_samples_split': [2, 3, 4, 5],
            'n_estimators': [100, 150, 200]}

clf = RandomForestClassifier(oob_score=True, random_state=42)

grid_search1 = GridSearchCV(clf, param_grid, scoring='roc_auc', cv=5, return_train_score=True, n_jobs=4)
grid_search1.fit(X_train.values, y_train.values)

cv_results1 = pd.DataFrame(grid_search1.cv_results_)
best_model = grid_search1.best_estimator_
best_model

GridSearchCV(cv=5,
             estimator=RandomForestClassifier(oob_score=True, random_state=42),
             n_jobs=4,
             param_grid={'max_features': [5, 6, 7, 8, 9, 10, 11, 12, 13, 14,
                                          15],
                         'min_samples_split': [2, 3, 4, 5],
                         'n_estimators': [100, 150, 200]},
             return_train_score=True, scoring='roc_auc')

RandomForestClassifier(max_features=7, n_estimators=200, oob_score=True,
                       random_state=42)

In [6]:
print(f"Best model oob score: {best_model.oob_score_}")
y_test_score = best_model.predict_proba(X_test)[:, 1]
print(f"ROC AUC: TEST {roc_auc_score(y_test, y_test_score):.6f}\n")

Best model oob score: 0.8270954138112809
ROC AUC: TEST 0.913444



In [7]:
cv_results1.loc[:, cols_to_view].sort_values('rank_test_score').head(10)

,rank_test_score,param_max_features,param_min_samples_split,param_n_estimators,mean_test_score,std_test_score,mean_train_score,std_train_score
26,1,7,2,200,0.89587749,0.01248500,1.00000000,0.00000000
62,2,10,2,200,0.89524363,0.01174297,1.00000000,0.00000000
25,3,7,2,150,0.89517077,0.01299246,1.00000000,0.00000000
76,4,11,3,150,0.89516528,0.01123649,0.99998714,0.00000534
24,5,7,2,100,0.89488549,0.01288568,1.00000000,0.00000000
50,6,9,2,200,0.89482350,0.01313602,1.00000000,0.00000000
48,7,9,2,100,0.89456435,0.01271117,1.00000000,0.00000000
49,8,9,2,150,0.89449143,0.01209344,1.00000000,0.00000000
13,9,6,2,150,0.89449127,0.01259369,1.00000000,0.00000000
2,10,5,2,200,0.89446100,0.01268410,1.00000000,0.00000000


In [8]:
featimp_df = pd.DataFrame({'feature_name': X.columns, 'importance': best_model.feature_importances_})
featimp_df.sort_values('importance', ascending=False).head(10)

,feature_name,importance
8,Bar,0.13280221
9,CoffeeHouse,0.04675748
12,Restaurant20To50,0.04235856
10,CarryAway,0.04228904
11,RestaurantLessThan20,0.04108017
7,minsToCouponDest,0.03058365
0,has_children,0.02981448
47,gender_Male,0.02129778
49,maritalStatus_Single,0.02032457
48,maritalStatus_Married partner,0.01893958


In [9]:
import pickle

with open('model/phase2/Bar_GridSearchCV_RFC.pickle', "wb") as f:
    pickle.dump(grid_search1,f)
    
with open('model/phase2/Bar_best_model_RFC.pickle', "wb") as f:
    pickle.dump(best_model,f)